In [53]:
#import all necessary libraries
%matplotlib inline
import requests
import pandas as pd
import numpy as np
import math
import glob
from datetime import datetime
from datetime import timedelta
from datetime import date

from epiweeks import Week, Year
from delphi_epidata import Epidata
from dateutil.relativedelta import relativedelta


In [54]:
daily_csv = pd.read_csv("covid-hospitalization-daily-all-state-merged_vEW202218.csv")
daily_csv

/Users/srikarbalusu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,epiweek,region,fips,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,...,Excess Estimate,death_jhu_cumulative,death_jhu_incidence,fb_survey_wcli,fb_survey_wili,positiveIncr_cumulative,positiveIncr,cdc_negativeIncr,cdc_positiveIncr,cdc_total_resultsIncr
0,2020-01-01,202001,X,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,202001,X,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,202001,X,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,202001,X,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,202002,X,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1549.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43753,2022-05-03,202218,WY,56,5.0,1.0,59.0,6.0,-12.0,1.0,...,NaN,1814.0,2.0,NaN,NaN,156745.0,195.0,455.0,13.0,469.0
43754,2022-05-04,202218,WY,56,0.0,-2.0,61.0,6.0,-14.0,1.0,...,NaN,1814.0,0.0,NaN,NaN,156745.0,0.0,770.0,24.0,800.0
43755,2022-05-05,202218,WY,56,5.0,2.0,112.0,13.0,-14.0,-1.0,...,NaN,1814.0,0.0,NaN,NaN,156745.0,0.0,481.0,22.0,505.0
43756,2022-05-06,202218,WY,56,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1814.0,0.0,NaN,NaN,156745.0,0.0,NaN,NaN,NaN


In [55]:
daily_csv.columns

Index(['date', 'epiweek', 'region', 'fips',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'apple_mobility',
       'cdc_hospitalized', 'Stage_One_Doses', 'Stage_Two_Doses',
       'smoothed_wcovid_vaccinated', 'smoothed_wtested_positive_14d',
       'smoothed_wwearing_mask_7d', 'smoothed_wtravel_outside_state_7d',
       'smoothed_wspent_time_indoors_1d', 'covidnet', 'Observed Number',
       'Excess Estimate', 'death_jhu_cumulative', 'death_jhu_incidence',
       'fb_survey_wcli', 'fb_survey_wili', 'positiveIncr_cumulative',
       'positiveIncr', 'cdc_negativeIncr', 'cdc_positiveIncr',
       'cdc_total_resultsIncr'],
      dtype='object')

In [56]:
mean_df = daily_csv.groupby(by=['epiweek','region'],as_index=False, sort=False).mean()
mean_df.head(50)

,epiweek,region,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,apple_mobility,cdc_hospitalized,...,Excess Estimate,death_jhu_cumulative,death_jhu_incidence,fb_survey_wcli,fb_survey_wili,positiveIncr_cumulative,positiveIncr,cdc_negativeIncr,cdc_positiveIncr,cdc_total_resultsIncr
0,202001,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.000000,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202002,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.714286,...,1549.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202003,X,NaN,NaN,NaN,NaN,NaN,NaN,109.531667,0.000000,...,1039.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202004,X,NaN,NaN,NaN,NaN,NaN,NaN,107.582857,0.000000,...,891.000000,1.000000,0.250000,NaN,NaN,1.500000e+00,0.500000,NaN,NaN,NaN
4,202005,X,NaN,NaN,NaN,NaN,NaN,NaN,106.434286,0.000000,...,877.000000,1.000000,0.000000,NaN,NaN,6.142857e+00,0.857143,NaN,NaN,NaN
5,202006,X,NaN,NaN,NaN,NaN,NaN,NaN,106.075714,0.000000,...,853.666667,1.000000,0.000000,NaN,NaN,1.100000e+01,0.571429,NaN,NaN,NaN
6,202007,X,6.000000,2.000000,15.000000,3.000000,2.000000,-1.000000,115.221429,0.000000,...,681.333333,1.000000,0.000000,NaN,NaN,1.314286e+01,0.285714,NaN,NaN,NaN
7,202008,X,3.571429,0.142857,12.714286,0.142857,-2.714286,0.714286,116.917143,0.000000,...,882.000000,1.000000,0.000000,NaN,NaN,1.557143e+01,0.285714,NaN,NaN,NaN
8,202009,X,6.857143,2.857143,12.714286,3.857143,2.428571,-0.714286,117.914286,0.000000,...,701.333333,1.142857,0.142857,NaN,NaN,1.914286e+01,1.428571,NaN,NaN,NaN
9,202010,X,8.857143,7.000000,18.142857,3.571429,3.000000,-1.000000,118.841429,0.000000,...,922.666667,10.714286,2.285714,NaN,NaN,1.622857e+02,55.857143,8.634286e+02,105.714286,9.730000e+02


In [57]:
sum_df = daily_csv.groupby(by=['epiweek','region'],as_index=False, sort=False).sum(min_count=1)
sum_df

,epiweek,region,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,apple_mobility,cdc_hospitalized,...,Excess Estimate,death_jhu_cumulative,death_jhu_incidence,fb_survey_wcli,fb_survey_wili,positiveIncr_cumulative,positiveIncr,cdc_negativeIncr,cdc_positiveIncr,cdc_total_resultsIncr
0,202001,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,396.0,...,4936.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202002,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264.0,...,10847.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202003,X,NaN,NaN,NaN,NaN,NaN,NaN,657.19,0.0,...,7277.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202004,X,NaN,NaN,NaN,NaN,NaN,NaN,753.08,0.0,...,6237.000000,4.0,1.0,NaN,NaN,6.0,2.0,NaN,NaN,NaN
4,202005,X,NaN,NaN,NaN,NaN,NaN,NaN,745.04,0.0,...,6139.000000,7.0,0.0,NaN,NaN,43.0,6.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6268,202214,WY,-21.0,-55.0,213.0,24.0,-83.0,6.0,1393.94,19.0,...,0.000000,12562.0,5.0,NaN,NaN,1093688.0,114.0,4549.0,100.0,4659.0
6269,202215,WY,-39.0,-40.0,68.0,-3.0,-98.0,18.0,NaN,10.0,...,0.000000,12597.0,5.0,NaN,NaN,1093852.0,23.0,4195.0,111.0,4321.0
6270,202216,WY,-64.0,-88.0,285.0,3.0,-107.0,11.0,NaN,8.0,...,0.000000,12637.0,6.0,NaN,NaN,1094476.0,134.0,2621.0,83.0,2710.0
6271,202217,WY,0.0,-30.0,391.0,67.0,-80.0,4.0,NaN,5.0,...,NaN,12674.0,5.0,NaN,NaN,1095534.0,158.0,3856.0,131.0,4002.0


In [58]:
last_df = daily_csv.groupby(by=['epiweek','region'],as_index=False, sort=False).last()
print(len(last_df))
last_df

6273


,epiweek,region,date,fips,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,...,Excess Estimate,death_jhu_cumulative,death_jhu_incidence,fb_survey_wcli,fb_survey_wili,positiveIncr_cumulative,positiveIncr,cdc_negativeIncr,cdc_positiveIncr,cdc_total_resultsIncr
0,202001,X,2020-01-04,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1234.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202002,X,2020-01-11,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1549.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202003,X,2020-01-18,US,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202004,X,2020-01-25,US,NaN,NaN,NaN,NaN,NaN,NaN,...,891.000000,1.0,0.0,NaN,NaN,2.0,0.0,NaN,NaN,NaN
4,202005,X,2020-02-01,US,NaN,NaN,NaN,NaN,NaN,NaN,...,877.000000,1.0,0.0,NaN,NaN,8.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6268,202214,WY,2022-04-09,56,-1.0,-7.0,34.0,13.0,-8.0,-1.0,...,0.000000,1796.0,0.0,NaN,NaN,156281.0,0.0,557.0,18.0,577.0
6269,202215,WY,2022-04-16,56,-4.0,6.0,29.0,3.0,-12.0,1.0,...,0.000000,1801.0,0.0,NaN,NaN,156258.0,0.0,429.0,16.0,447.0
6270,202216,WY,2022-04-23,56,-22.0,-23.0,-17.0,-16.0,-16.0,7.0,...,0.000000,1807.0,0.0,NaN,NaN,156392.0,0.0,237.0,13.0,251.0
6271,202217,WY,2022-04-30,56,2.0,-3.0,70.0,16.0,-7.0,-1.0,...,NaN,1812.0,0.0,NaN,NaN,156550.0,0.0,246.0,8.0,255.0


In [59]:
weekly_df = pd.DataFrame()
mean_key = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'apple_mobility','smoothed_wcovid_vaccinated', 'smoothed_wtested_positive_14d',
       'smoothed_wwearing_mask_7d', 'smoothed_wtravel_outside_state_7d',
       'smoothed_wspent_time_indoors_1d','fb_survey_wcli', 'fb_survey_wili']

sum_key = ['cdc_hospitalized', 'death_jhu_incidence',
       'positiveIncr', 'cdc_negativeIncr', 'cdc_positiveIncr',
       'cdc_total_resultsIncr']

last_key = ['Stage_One_Doses', 'Stage_Two_Doses','covidnet', 'Observed Number','Excess Estimate',
            'death_jhu_cumulative','positiveIncr_cumulative']
for ix, row in sum_df.iterrows(): 
    add_dict = {}
    add_dict['epiweek'] = sum_df['epiweek'][ix]
    print(sum_df['epiweek'][ix])
    add_dict['region'] = sum_df['region'][ix]
    for i in mean_key: 
        add_dict[i] = mean_df[i][ix]
    for i in sum_key: 
        add_dict[i] = sum_df[i][ix]
    for i in last_key: 
        add_dict[i] = last_df[i][ix]
    weekly_df = weekly_df.append(add_dict, ignore_index = True)

202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020

202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148

202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111

202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102

202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033

202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202013
202014
202015
202016
202017
202018
202019
202020
202021
202022
202023
202024
202025
202026
202027
202028
202029
202030
202031
202032
202033
202034
202035
202036
202037
202038
202039
202040
202041
202042
202043
202044
202045
202046
202047
202048
202049
202050
202051
202052
202053
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214
202215
202216
202217
202218
202001
202002
202003
202004
202005
202006
202007

In [61]:
weekly_df = weekly_df.reindex(columns=['epiweek', 'region',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'apple_mobility',
       'cdc_hospitalized', 'Stage_One_Doses', 'Stage_Two_Doses',
       'smoothed_wcovid_vaccinated', 'smoothed_wtested_positive_14d',
       'smoothed_wwearing_mask_7d', 'smoothed_wtravel_outside_state_7d',
       'smoothed_wspent_time_indoors_1d', 'covidnet', 'Observed Number',
       'Excess Estimate', 'death_jhu_cumulative', 'death_jhu_incidence',
       'fb_survey_wcli', 'fb_survey_wili', 'positiveIncr_cumulative',
       'positiveIncr', 'cdc_negativeIncr', 'cdc_positiveIncr',
       'cdc_total_resultsIncr'])
weekly_df['epiweek'] = weekly_df['epiweek'].astype('int')
weekly_df.to_csv("dailytoweeklytest.csv",index=False)

In [60]:
weekly_df.head(50)

,Excess Estimate,Observed Number,Stage_One_Doses,Stage_Two_Doses,apple_mobility,cdc_hospitalized,cdc_negativeIncr,cdc_positiveIncr,cdc_total_resultsIncr,covidnet,...,region,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,smoothed_wcovid_vaccinated,smoothed_wspent_time_indoors_1d,smoothed_wtested_positive_14d,smoothed_wtravel_outside_state_7d,smoothed_wwearing_mask_7d,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1234.000000,179143.000000,NaN,NaN,NaN,396.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1549.666667,180831.000000,NaN,NaN,NaN,264.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1039.666667,176752.000000,NaN,NaN,109.531667,0.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,891.000000,176066.000000,NaN,NaN,107.582857,0.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,877.000000,175092.000000,NaN,NaN,106.434286,0.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,853.666667,177040.000000,NaN,NaN,106.075714,0.0,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,681.333333,175040.000000,NaN,NaN,115.221429,0.0,NaN,NaN,NaN,NaN,...,X,-1.000000,6.000000,NaN,NaN,NaN,NaN,NaN,3.000000,2.000000
7,882.000000,175393.000000,NaN,NaN,116.917143,0.0,NaN,NaN,NaN,NaN,...,X,0.714286,3.571429,NaN,NaN,NaN,NaN,NaN,0.142857,-2.714286
8,701.333333,176648.000000,NaN,NaN,117.914286,0.0,NaN,NaN,NaN,NaN,...,X,-0.714286,6.857143,NaN,NaN,NaN,NaN,NaN,3.857143,2.428571
9,922.666667,177707.000000,NaN,NaN,118.841429,0.0,6044.0,740.0,6811.0,0.1,...,X,-1.000000,8.857143,NaN,NaN,NaN,NaN,NaN,3.571429,3.000000


In [22]:
sum_df['epiweek'][100]

202148

In [12]:
weekly_df = pd.DataFrame()
d_dict = {'First Name': 'Vikram', 'Last Name': 'Aruchamy', 'Country': 'India'}
weekly_df = weekly_df.append(d_dict, ignore_index = True)
weekly_df

,Country,First Name,Last Name
0,India,Vikram,Aruchamy


In [21]:
daily_csv['epiweek'][100]

202015

In [33]:
flag_dict = {}
for ix, row in daily_csv.iterrows():
    epiweek_dict = {}
    curr_epi = row['epiweek']
    move = curr_epi
    counter = 0
    if move in flag_dict.keys(): 
        print("SAUCE")
        continue
    while move == curr_epi: 
        move = daily_csv['epiweek'][ix+counter]
        counter += 1 
    flag_dict[daily_csv['epiweek'][ix]] = True
    print(ix)
    print(move)
    print(flag_dict.keys())

0
202002
dict_keys([202001])
SAUCE
SAUCE
SAUCE
4
202003
dict_keys([202001, 202002])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
11
202004
dict_keys([202001, 202002, 202003])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
18
202005
dict_keys([202001, 202002, 202003, 202004])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
25
202006
dict_keys([202001, 202002, 202003, 202004, 202005])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
32
202007
dict_keys([202001, 202002, 202003, 202004, 202005, 202006])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
39
202008
dict_keys([202001, 202002, 202003, 202004, 202005, 202006, 202007])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
46
202009
dict_keys([202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
53
202010
dict_keys([202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
60
202011
dict_keys([202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010])
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE


SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

...

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUC

SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE
SAUCE


In [ ]:
if date.today().weekday() != 6:
    week_num = (date.today()-timedelta(2)).strftime("%U")
    year_week_num = "2022"  + week_num
    print(year_week_num)
    week_end_date = (date.today() +timedelta((5-date.today().weekday()) % 7 )).strftime('%Y-%m-%d')
    week_end_date = (date.today() - timedelta(2)).strftime('%Y-%m-%d')
    week_end_string = (date.today() + timedelta((5-date.today().weekday()) % 7 )).strftime('%Y%m%d')
    week_end_string = (date.today() - timedelta(2)).strftime('%Y%m%d')

else:
    week_num = (date.today()-timedelta(days=1)).strftime("%U")
    year_week_num = "2022" + week_num
    print(year_week_num)
    week_end_date = (date.today() -timedelta((date.today().weekday()-5) % 7 )).strftime('%Y-%m-%d')
    week_end_string = (date.today() - timedelta((date.today().weekday()-5) % 7 )).strftime('%Y%m%d')
### INPUT FILE DIRECTORY #####
# data_path="/Users/anikat/Downloads/covid-hospitalization-data/"
data_path="./"
# kinsa_path= "/Users/anikat/Downloads/kinsahealth/"
kinsa_path = "./"
### INPUT: EPIWEEK START, END WEEK ####
#Both the method args will be same. This is to keep last date in the ouput file to same as epiweek
epiweek1,epiweek_date1=get_epiweek_list('202001','202053',2020)
epiweek2,epiweek_date2=get_epiweek_list('202101','202152',2021)
epiweek3,epiweek_date3=get_epiweek_list('202201',year_week_num,2022)




epiweek=epiweek1+epiweek2+epiweek3
epiweek_date=epiweek_date1+epiweek_date2+epiweek_date3
print('CDC hospitalization')
end_week=int(week_num) #2021
cdc_hosp,cols_cdc,flu_hosp,cols_flu=read_cdc_hosp(data_path,epiweek_date,state_index,end_week)